In [1]:
from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
import json
from collections import defaultdict

# Conexión al clúster de Cassandra con los nombres de host de tus nodos
cluster = Cluster(['127.0.0.1'], port=9042)#, auth_provider=auth_provider)

# Crear una sesión
session = cluster.connect()

# Seleccionar el keyspace (si ya tienes uno creado)
session.set_keyspace('videojuego')

## Lectura hall of fame

In [ ]:
def get_hall_of_fame(country: str):
    # Establecer consistencia más fuerte
    # Para asegurar la máxima consistencia usamos ALL. Todas las réplicas tendrán que responder a la query
    session.default_consistency_level = ConsistencyLevel.ALL

    # Query 
    query = """
        SELECT Mazmorra_id, Nombre_mazmorra, Email, Nombre_usuario, Tiempo, Fecha 
        FROM Hall_of_fame 
        WHERE Pais = %s
        ALLOW FILTERING;
    """
    
    rows = session.execute(query, (country, ))
    
    # Organizar datos en un diccionario agrupado por Mazmorra_id
    dungeon_dict = defaultdict(lambda: {"dungeon_id": 0, "dungeon_name": "", "Top_5": []})

    for row in rows:
        dungeon_id = row.mazmorra_id
        dungeon_dict[dungeon_id]["dungeon_id"] = dungeon_id
        dungeon_dict[dungeon_id]["dungeon_name"] = row.nombre_mazmorra
        dungeon_dict[dungeon_id]["Top_5"].append({
            "email": row.email,
            "user_name": row.nombre_usuario,
            "time_minutes": row.tiempo,
            "date": row.fecha
        })
    
    result = json.dumps(list(dungeon_dict.values()), indent=4) 
    return result


In [22]:
# Ejemplo de uso
country = "ja_JP"
top_players = get_hall_of_fame(country)
print(top_players)


[
    {
        "dungeon_id": 19,
        "dungeon_name": "Dalhylles, Culverts of the Grumpy Bandits",
        "Top_5": [
            {
                "email": "aabe@example.net",
                "user_name": "minorusuzuki",
                "time_minutes": 1.0,
                "date": "2020-03-28 22:37:31"
            }
        ]
    },
    {
        "dungeon_id": 10,
        "dungeon_name": "Corriedal, Sewers of the Terrible Emperors",
        "Top_5": [
            {
                "email": "abeshohei@example.net",
                "user_name": "haruka48",
                "time_minutes": 1.0,
                "date": "2021-11-07 17:24:04"
            },
            {
                "email": "aabe@example.net",
                "user_name": "minorusuzuki",
                "time_minutes": 1.0,
                "date": "2020-08-06 19:43:39"
            }
        ]
    },
    {
        "dungeon_id": 17,
        "dungeon_name": "Marshgreat, Catacombs of the Wandering Degenerates",
        

## Lectura Statistics

In [26]:
def get_user_statistics(user_email: str, dungeon_id: int):
    # Establecer consistencia más fuerte
    session.default_consistency_level = ConsistencyLevel.ALL
    
    # Query 
    query = """
        SELECT Tiempo, Fecha 
        FROM Statistic 
        WHERE Email = %s AND Mazmorra_id = %s;
    """
    
    rows = session.execute(query, (user_email, dungeon_id))
    
    # converitr a lista de diccionarios
    result = [{"time_minute": row.tiempo, "date": row.fecha} for row in rows]
    
    # Convertir a JSON
    return json.dumps(result, indent=4)


In [27]:
# Ejemplo de uso
user_email = "abag@example.com"
dungeon_id = 1
statistics_json = get_user_statistics(user_email, dungeon_id)
print(statistics_json)


[
    {
        "time_minute": 1.0,
        "date": "2022-11-27 23:07:39"
    },
    {
        "time_minute": 4.0,
        "date": "2022-09-05 03:38:31"
    },
    {
        "time_minute": 8.0,
        "date": "2022-03-14 23:10:56"
    },
    {
        "time_minute": 9.0,
        "date": "2022-05-07 12:59:48"
    },
    {
        "time_minute": 10.0,
        "date": "2022-10-11 22:50:12"
    },
    {
        "time_minute": 13.0,
        "date": "2022-11-14 01:56:46"
    },
    {
        "time_minute": 14.0,
        "date": "2022-08-30 13:10:43"
    },
    {
        "time_minute": 15.0,
        "date": "2022-05-22 17:56:39"
    },
    {
        "time_minute": 17.0,
        "date": "2022-12-11 08:56:01"
    },
    {
        "time_minute": 19.0,
        "date": "2022-10-06 18:48:44"
    }
]


## Lectura Top horde

In [34]:
def get_top_horde(country: str, event_id: int, k: int):
    # Reducimos consistencia para asegurar velocidad en las lecturas
    session.default_consistency_level = ConsistencyLevel.LOCAL_ONE

    # Query optimizada
    query = """
        SELECT Email, Nombre_usuario, N_killed 
        FROM Top_horde 
        WHERE Evento_id = %s AND Pais = %s 
        LIMIT %s;
    """
    
    rows = session.execute(query, (event_id, country, k))
    
    # Convertir a lista de diccionarios
    result = [
        {
            "posicion": idx + 1,  # Asignamos un ranking (opcional)
            "user_name": row.nombre_usuario,
            "email": row.email,
            "n_killed": row.n_killed
        }
        for idx, row in enumerate(rows)
    ]

    # Convertir a JSON
    return json.dumps(result, indent=4)


In [35]:
# Ejemplo de uso
country = "ja_JP"
event_id = 2
k = 3
top_horde_json = get_top_horde(country, event_id, k)
print(top_horde_json)


[
    {
        "posicion": 1,
        "user_name": "ukobayashi",
        "email": "jyamashita@example.org",
        "n_killed": 22
    },
    {
        "posicion": 2,
        "user_name": "yasuhiro13",
        "email": "nakagawanaoto@example.org",
        "n_killed": 21
    },
    {
        "posicion": 3,
        "user_name": "yumiko56",
        "email": "ishikawarei@example.org",
        "n_killed": 20
    }
]


## Cerramos conexión

In [ ]:
# Cerrar la conexión
cluster.shutdown()